In [1]:
import csv
import neo4j
import json
import math
import numpy as np
import pandas as pd
from geographiclib.geodesic import Geodesic
import psycopg2

In [2]:
#
# function to run a select query and return rows in a pandas dataframe
# pandas puts all numeric values from postgres to float
# if it will fit in an integer, change it to integer
#

def my_select_query_pandas(query, rollback_before_flag, rollback_after_flag):
    "function to run a select query and return rows in a pandas dataframe"
    
    if rollback_before_flag:
        connection.rollback()
    
    df = pd.read_sql_query(query, connection)
    
    if rollback_after_flag:
        connection.rollback()
    
    # fix the float columns that really should be integers
    
    for column in df:
    
        if df[column].dtype == "float64":

            fraction_flag = False

            for value in df[column].values:
                
                if not np.isnan(value):
                    if value - math.floor(value) != 0:
                        fraction_flag = True

            if not fraction_flag:
                df[column] = df[column].astype('Int64')
    
    return(df)
    

In [3]:
def my_neo4j_run_query_pandas(query, **kwargs):
    "run a query and return the results in a pandas dataframe"
    
    result = session.run(query, **kwargs)
    
    df = pd.DataFrame([r.values() for r in result], columns=result.keys())
    
    return df

In [4]:
driver = neo4j.GraphDatabase.driver(uri="neo4j://neo4j:7687", auth=("neo4j","ucb_mids_w205"))

In [5]:
session = driver.session(database="neo4j")

In [6]:
def my_neo4j_wipe_out_database():
    "wipe out database by deleting all nodes and relationships"
    
    query = "match (node)-[relationship]->() delete node, relationship"
    session.run(query)
    
    query = "match (node) delete node"
    session.run(query)

In [7]:
def my_neo4j_number_nodes_relationships():
    "print the number of nodes and relationships"
   
    
    query = """
        match (n) 
        return n.name as node_name, labels(n) as labels
        order by n.name
    """
    
    df = my_neo4j_run_query_pandas(query)
    
    number_nodes = df.shape[0]
    
    
    query = """
        match (n1)-[r]->(n2) 
        return n1.name as node_name_1, labels(n1) as node_1_labels, 
            type(r) as relationship_type, n2.name as node_name_2, labels(n2) as node_2_labels
        order by node_name_1, node_name_2
    """
    
    df = my_neo4j_run_query_pandas(query)
    
    number_relationships = df.shape[0]
    
    print("-------------------------")
    print("  Nodes:", number_nodes)
    print("  Relationships:", number_relationships)
    print("-------------------------")


In [8]:
def my_neo4j_create_node(station_name):
    "create a node with label Station"
    
    query = """
    
    CREATE (:Station {name: $station_name})
    
    """
    
    session.run(query, station_name=station_name)
    

In [9]:
def my_neo4j_create_relationship_one_way(from_station, to_station, weight):
    "create a relationship one way between two stations with a weight"
    
    query = """
    
    MATCH (from:Station), 
          (to:Station)
    WHERE from.name = $from_station and to.name = $to_station
    CREATE (from)-[:LINK {weight: $weight}]->(to)
    
    """
    
    session.run(query, from_station=from_station, to_station=to_station, weight=weight)
    

In [10]:
def my_neo4j_create_relationship_two_way(from_station, to_station, weight):
    "create relationships two way between two stations with a weight"
    
    query = """
    
    MATCH (from:Station), 
          (to:Station)
    WHERE from.name = $from_station and to.name = $to_station
    CREATE (from)-[:LINK {weight: $weight}]->(to),
           (to)-[:LINK {weight: $weight}]->(from)
    
    """
    
    session.run(query, from_station=from_station, to_station=to_station, weight=weight)
    

In [11]:
connection = psycopg2.connect(
    user = "postgres",
    password = "ucb",
    host = "postgres",
    port = "5432",
    database = "postgres"
)

In [12]:
cursor = connection.cursor()

## Wiping Out the Neo4j

In [13]:
my_neo4j_wipe_out_database()

In [14]:
my_neo4j_number_nodes_relationships()

-------------------------
  Nodes: 0
  Relationships: 0
-------------------------


In [15]:
connection.rollback()

query = """

select station
from stations
order by station

"""

cursor.execute(query)

connection.rollback()

rows = cursor.fetchall()

for row in rows:
    
    station = row[0]
    
    my_neo4j_create_node('depart ' + station)
    my_neo4j_create_node('arrive ' + station)
    

In [16]:
connection.rollback()

query = """

select station, line
from lines
order by station, line

"""

cursor.execute(query)

connection.rollback()

rows = cursor.fetchall()

for row in rows:
    
    station = row[0]
    line = row[1]
    
    depart = 'depart ' + station
    arrive = 'arrive ' + station
    line_station = line + ' ' + station
    
    my_neo4j_create_node(line_station)
    my_neo4j_create_relationship_one_way(depart, line_station, 0)
    my_neo4j_create_relationship_one_way(line_station, arrive, 0)
    

In [17]:
connection.rollback()

query = """

select a.station, a.line as from_line, b.line as to_line, s.transfer_time
from lines a
     join lines b
       on a.station = b.station and a.line <> b.line 
     join stations s
       on a.station = s.station
order by 1, 2, 3

"""

cursor.execute(query)

connection.rollback()

rows = cursor.fetchall()

for row in rows:
    
    station = row[0]
    from_line = row[1]
    to_line = row[2]
    transfer_time = int(row[3])
    
    from_station = from_line + ' ' + station
    to_station = to_line + ' ' + station
    
    my_neo4j_create_relationship_one_way(from_station, to_station, transfer_time)
    

In [18]:
connection.rollback()

query = """

select a.line, a.station as from_station, b.station as to_station, t.travel_time
from lines a
  join lines b
    on a.line = b.line and b.sequence = (a.sequence + 1)
  join travel_times t
    on (a.station = t.station_1 and b.station = t.station_2)
        or (a.station = t.station_2 and b.station = t.station_1)
order by line, from_station, to_station

"""

cursor.execute(query)

connection.rollback()

rows = cursor.fetchall()

for row in rows:
    
    line = row[0]
    from_station = line + ' ' + row[1]
    to_station = line + ' ' + row[2]
    travel_time = int(row[3])
    
    my_neo4j_create_relationship_two_way(from_station, to_station, travel_time)

In [19]:
my_neo4j_number_nodes_relationships()

-------------------------
  Nodes: 214
  Relationships: 652
-------------------------


In [20]:
def my_neo4j_shortest_path(from_station, to_station):
    "given a from station and to station, run and print the shortest path"
    
    query = "CALL gds.graph.drop('ds_graph', false) yield graphName"
    session.run(query)

    query = "CALL gds.graph.project('ds_graph', 'Station', 'LINK', {relationshipProperties: 'weight'})"
    session.run(query)

    query = """

    MATCH (source:Station {name: $source}), (target:Station {name: $target})
    CALL gds.shortestPath.dijkstra.stream(
        'ds_graph', 
        { sourceNode: source, 
          targetNode: target, 
          relationshipWeightProperty: 'weight'
        }
    )
    YIELD index, sourceNode, targetNode, totalCost, nodeIds, costs, path
    RETURN
        gds.util.asNode(sourceNode).name AS from,
        gds.util.asNode(targetNode).name AS to,
        totalCost,
        [nodeId IN nodeIds | gds.util.asNode(nodeId).name] AS nodes,
        costs
    ORDER BY index

    """

    result = session.run(query, source=from_station, target=to_station)
    
    for r in result:
        
        total_cost = int(r['totalCost'])
        
        print("\n--------------------------------")
        print("   Total Cost: ", total_cost)
        print("   Minutes: ", round(total_cost / 60.0,1))
        print("--------------------------------")
        
        nodes = r['nodes']
        costs = r['costs']
        
        i = 0
        previous = 0
        
        for n in nodes:
            
            print(n + ", " + str(int(costs[i]) - previous)  + ", " + str(int(costs[i])))
            
            previous = int(costs[i])
            i += 1
    

In [21]:
def my_calculate_box(point, miles):
    "Given a point and miles, calculate the box in form left, right, top, bottom"
    
    geod = Geodesic.WGS84

    kilometers = miles * 1.60934
    meters = kilometers * 1000

    g = geod.Direct(point[0], point[1], 270, meters)
    left = (g['lat2'], g['lon2'])

    g = geod.Direct(point[0], point[1], 90, meters)
    right = (g['lat2'], g['lon2'])

    g = geod.Direct(point[0], point[1], 0, meters)
    top = (g['lat2'], g['lon2'])

    g = geod.Direct(point[0], point[1], 180, meters)
    bottom = (g['lat2'], g['lon2'])
    
    return(left, right, top, bottom)

In [22]:
def my_station_get_zips(station, miles):
    "given a station, pull all zip codes with miles distance, print them, sum the population"
    
    connection.rollback()
    
    query = "select latitude, longitude from stations "
    query += "where station = '" + station + "'"
    
    cursor.execute(query)
    
    connection.rollback()
    
    rows = cursor.fetchall()
    
    for row in rows:
        latitude = row[0]
        longitude = row[1]
        
    point = (latitude, longitude)
        
    (left, right, top, bottom) = my_calculate_box(point, miles)
    
    query = "select zip, population from zip_codes "
    query += " where latitude >= " + str(bottom[0])
    query += " and latitude <= " + str(top [0])
    query += " and longitude >= " + str(left[1])
    query += " and longitude <= " + str(right[1])
    query += " order by 1 "

    cursor.execute(query)
    
    connection.rollback()
    
    rows = cursor.fetchall()
    
    print("\n-------------------------------------------------------------------------------")
    print("  Zip Codes within " + str(miles) + " mile(s) of " + station + " BART Station")
    print("-------------------------------------------------------------------------------\n")
    
    total_population = 0
    
    for row in rows:
        zip = row[0]
        population = row[1]
        print("     zip:", zip, "  population: ", f'{population:10,}')
        total_population += population
        
    
    print("\n-------------------------------------------------------------------------------")
    print("  Total Population: ", f'{total_population:10,}')
    print("-------------------------------------------------------------------------------")

In [23]:
my_neo4j_shortest_path('depart Dublin', 'arrive Antioch')


--------------------------------
   Total Cost:  5813
   Minutes:  96.9
--------------------------------
depart Dublin, 0, 0
blue Dublin, 0, 0
blue West Dublin, 180, 180
blue Castro Valley, 600, 780
blue Bay Fair, 240, 1020
blue San Leandro, 240, 1260
blue Coliseum, 240, 1500
orange Coliseum, 54, 1554
orange Fruitvale, 240, 1794
orange Lake Merritt, 300, 2094
orange 12th Street, 180, 2274
orange 19th Street, 120, 2394
orange MacArthur, 180, 2574
yellow MacArthur, 59, 2633
yellow Rockridge, 240, 2873
yellow Orinda, 300, 3173
yellow Lafayette, 300, 3473
yellow Walnut Creek, 300, 3773
yellow Pleasant Hill, 120, 3893
yellow Concord, 360, 4253
yellow North Concord, 180, 4433
yellow Pittsburg, 360, 4793
yellow Pittsburg Center, 600, 5393
yellow Antioch, 420, 5813
arrive Antioch, 0, 5813


In [24]:
my_station_get_zips('Downtown Berkeley', 1)


-------------------------------------------------------------------------------
  Zip Codes within 1 mile(s) of Downtown Berkeley BART Station
-------------------------------------------------------------------------------

     zip: 94702   population:      17,092
     zip: 94703   population:      21,937
     zip: 94704   population:      29,190
     zip: 94709   population:      11,740
     zip: 94720   population:       2,971

-------------------------------------------------------------------------------
  Total Population:      82,930
-------------------------------------------------------------------------------


In [25]:
def connect_store_to_bart_network(store_name, store_coords):
    """
    Connect the store (as a Store node) to the nearest BART station's 'depart' and 'arrive' nodes in the graph,
    creating two-way relationships for bidirectional pathfinding.
    """
    # Step 1: Add the store node with the label `Store`
    query = """
    MERGE (:Store {name: $store_name})
    """
    session.run(query, store_name=store_name)

    # Step 2: Retrieve all BART stations with their coordinates
    query = """
    MATCH (station:Station)
    WHERE station.name STARTS WITH 'depart '
    RETURN station.name AS depart_station_name
    """
    df_depart_stations = my_neo4j_run_query_pandas(query)

    # Prepare to find the closest station
    geod = Geodesic.WGS84

    min_distance_minutes = None
    closest_depart_station_name = None
    closest_arrive_station_name = None

    for _, row in df_depart_stations.iterrows():
        bart_depart_station_name = row['depart_station_name']
        bart_station = bart_depart_station_name.replace('depart ', '')

        # Get BART station coordinates from PostgreSQL
        query = "SELECT latitude, longitude FROM stations WHERE station = %s"
        cursor.execute(query, (bart_station,))
        bart_coords = cursor.fetchone()

        if bart_coords:
            bart_lat, bart_lon = bart_coords
            store_lat, store_lon = store_coords

            # Calculate distance between store and BART station
            result = geod.Inverse(store_lat, store_lon, bart_lat, bart_lon)
            distance_meters = result['s12']
            distance_minutes = distance_meters / 80  # Assuming average walking speed of 80 meters/min

            # Update minimum distance
            if (min_distance_minutes is None) or (distance_minutes < min_distance_minutes):
                min_distance_minutes = distance_minutes
                closest_depart_station_name = bart_depart_station_name
                closest_arrive_station_name = 'arrive ' + bart_station

    # Step 3: Create relationships from the store to 'depart' node and from 'arrive' node to the store
    if closest_depart_station_name and closest_arrive_station_name:
        # Relationship from store to 'depart' node
        query = """
        MATCH (store:Store {name: $store_name}), (station:Station {name: $depart_station_name})
        MERGE (store)-[:LINK {weight: $distance_minutes}]->(station)
        """
        session.run(query, store_name=store_name, depart_station_name=closest_depart_station_name, distance_minutes=min_distance_minutes)

        # Relationship from 'arrive' node to store
        query = """
        MATCH (station:Station {name: $arrive_station_name}), (store:Store {name: $store_name})
        MERGE (station)-[:LINK {weight: $distance_minutes}]->(store)
        """
        session.run(query, arrive_station_name=closest_arrive_station_name, store_name=store_name, distance_minutes=min_distance_minutes)

        print(f"Connected {store_name} to {closest_depart_station_name} and {closest_arrive_station_name} to {store_name} with distance {min_distance_minutes:.2f} minutes.")
    else:
        print("No closest station found.")

In [26]:
def calculate_shortest_path_from_store(store_name, target_station):
    """
    Calculate the shortest path from the store to a specific BART station using travel time.
    """
    # Step 1: Drop and project the graph
    session.run("CALL gds.graph.drop('ds_graph', false) YIELD graphName")
    session.run("""
        CALL gds.graph.project(
            'ds_graph',
            ['Station', 'Store'],
            {
                LINK: {
                    type: 'LINK',
                    properties: 'weight'
                }
            }
        )
    """)

    # Step 2: Run Dijkstra's algorithm from the store to the target station
    query = """
    MATCH (source:Store {name: $store_name}), (target:Station {name: $target_station})
    CALL gds.shortestPath.dijkstra.stream(
        'ds_graph', 
        {
            sourceNode: source,
            targetNode: target,
            relationshipWeightProperty: 'weight'
        }
    )
    YIELD index, sourceNode, targetNode, totalCost, nodeIds, costs
    RETURN
        gds.util.asNode(sourceNode).name AS from,
        gds.util.asNode(targetNode).name AS to,
        totalCost,
        [nodeId IN nodeIds | gds.util.asNode(nodeId).name] AS nodes,
        costs
    """
    result = session.run(query, store_name=store_name, target_station=target_station)
    
    # Step 3: Display the results
    for record in result:
        print("\n--------------------------------")
        print(f"Shortest Path from {record['from']} to {record['to']}:")
        print(f"   Total Cost (Seconds): {round(record['totalCost'], 2)}")
        print(f"   Approx. Minutes: {round(record['totalCost'] / 60.0, 1)}")
        print("--------------------------------")

        previous_cost = 0
        for i, node in enumerate(record['nodes']):
            step_cost = record['costs'][i] - previous_cost if i < len(record['costs']) else 0
            cumulative_cost = record['costs'][i] if i < len(record['costs']) else record['totalCost']
            print(f"   {node} (Step Cost: {round(step_cost, 2)}, Cumulative Cost: {round(cumulative_cost, 2)})")
            previous_cost = cumulative_cost


In [27]:
def calculate_shortest_path_to_store(start_station, store_name):
    """
    Calculate the shortest path from a BART station to the store using travel time.
    """
    # Step 1: Drop and project the graph
    session.run("CALL gds.graph.drop('ds_graph', false) YIELD graphName")
    session.run("""
        CALL gds.graph.project(
            'ds_graph',
            ['Station', 'Store'],
            {
                LINK: {
                    type: 'LINK',
                    properties: 'weight'
                }
            }
        )
    """)

    # Step 2: Run Dijkstra's algorithm from the start station to the store
    query = """
    MATCH (source:Station {name: $start_station}), (target:Store {name: $store_name})
    CALL gds.shortestPath.dijkstra.stream(
        'ds_graph', 
        {
            sourceNode: source,
            targetNode: target,
            relationshipWeightProperty: 'weight'
        }
    )
    YIELD index, sourceNode, targetNode, totalCost, nodeIds, costs
    RETURN
        gds.util.asNode(sourceNode).name AS from,
        gds.util.asNode(targetNode).name AS to,
        totalCost,
        [nodeId IN nodeIds | gds.util.asNode(nodeId).name] AS nodes,
        costs
    """
    result = session.run(query, start_station=start_station, store_name=store_name)
    
    # Step 3: Display the results
    for record in result:
        print("\n--------------------------------")
        print(f"Shortest Path from {record['from']} to {record['to']}:")
        print(f"   Total Cost (Seconds): {round(record['totalCost'], 2)}")
        print(f"   Approx. Minutes: {round(record['totalCost'] / 60.0, 1)}")
        print("--------------------------------")

        previous_cost = 0
        for i, node in enumerate(record['nodes']):
            step_cost = record['costs'][i] - previous_cost if i < len(record['costs']) else 0
            cumulative_cost = record['costs'][i] if i < len(record['costs']) else record['totalCost']
            print(f"   {node} (Step Cost: {round(step_cost, 2)}, Cumulative Cost: {round(cumulative_cost, 2)})")
            previous_cost = cumulative_cost


In [28]:
# Coordinates of the store (latitude, longitude)
store_coords = (37.702152, -121.935791)  # Example coordinates near Dublin/Pleasanton

# Name of the store
store_name = "Acme Gourmet Meals"

# Connect the store to the nearest BART station
connect_store_to_bart_network(store_name, store_coords)

Connected Acme Gourmet Meals to depart West Dublin and arrive West Dublin to Acme Gourmet Meals with distance 8.94 minutes.


In [29]:
# Target station (e.g., 'arrive Antioch')
target_station = 'arrive Dublin'

# Calculate the shortest path from the store to the target station
calculate_shortest_path_from_store(store_name, target_station)


--------------------------------
Shortest Path from Acme Gourmet Meals to arrive Dublin:
   Total Cost (Seconds): 188.94
   Approx. Minutes: 3.1
--------------------------------
   Acme Gourmet Meals (Step Cost: 0.0, Cumulative Cost: 0.0)
   depart West Dublin (Step Cost: 8.94, Cumulative Cost: 8.94)
   blue West Dublin (Step Cost: 0.0, Cumulative Cost: 8.94)
   blue Dublin (Step Cost: 180.0, Cumulative Cost: 188.94)
   arrive Dublin (Step Cost: 0.0, Cumulative Cost: 188.94)


In [30]:
# Start station (e.g., 'depart Antioch')
start_station = 'depart Dublin'

# Calculate the shortest path from the start station to the store
calculate_shortest_path_to_store(start_station, store_name)


--------------------------------
Shortest Path from depart Dublin to Acme Gourmet Meals:
   Total Cost (Seconds): 188.94
   Approx. Minutes: 3.1
--------------------------------
   depart Dublin (Step Cost: 0.0, Cumulative Cost: 0.0)
   blue Dublin (Step Cost: 0.0, Cumulative Cost: 0.0)
   blue West Dublin (Step Cost: 180.0, Cumulative Cost: 180.0)
   arrive West Dublin (Step Cost: 0.0, Cumulative Cost: 180.0)
   Acme Gourmet Meals (Step Cost: 8.94, Cumulative Cost: 188.94)
